<a href="https://colab.research.google.com/github/Elwing-Chou/ml0602/blob/main/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0602/raw/main/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0602/raw/main/poem_test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f34b6b46410>)

In [3]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")

In [4]:
# 分類: 輸出 0 1 2 3
# series.replace(字典)
# {"李白":0, "杜甫":1, "白居易":2}
# series.unique/series.value_counts
u = train["作者"].unique()
name2index = {n:i for i, n in enumerate(u)}
index2name = {i:n for i, n in enumerate(u)}
print(name2index)
print(index2name)
y_train = train["作者"].replace(name2index)
y_test = test["作者"].replace(name2index)
y_test

{'李白': 0, '杜甫': 1, '白居易': 2}
{0: '李白', 1: '杜甫', 2: '白居易'}


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    2
21    2
22    2
23    2
24    2
25    2
26    2
27    2
28    2
29    2
Name: 作者, dtype: int64

In [ ]:
# series.apply(func不加小括號)
# !!我做過實驗了, 詩詞分類載入大辭典反而適得其反, 但是普通時候你要載
import jieba
def poemcut(p):
    return " ".join(jieba.cut(p))
x_train = train["內容"].apply(poemcut)
x_test = test["內容"].apply(poemcut)
x_test

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(x_train)
len(vec.vocabulary_)

52294

In [24]:
# fit_transform: 測試資料只做transform
# 稀疏矩陣
x_train_count = vec.transform(x_train)
x_test_count = vec.transform(x_test)
x_test_count

<30x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 352 stored elements in Compressed Sparse Row format>

In [30]:
# 檢查標點符號, \r \n
# vec.vocabulary_["。"]
# vec.vocabulary_["\r\n"]

In [31]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [33]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8

In [40]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
             columns=["{}(預測)".format(n) for n in u],
             index=["{}(正確)".format(n) for n in u])

,李白(預測),杜甫(預測),白居易(預測)
李白(正確),8,1,1
杜甫(正確),1,8,1
白居易(正確),1,1,8


In [52]:
poem = input("輸入詩:")
poems = [poemcut(poem)]
poems_count = vec.transform(poems)
pre = clf.predict(poems_count)[0]
print("答案是:", index2name[pre])
pre_proba = clf.predict_proba(poems_count)[0]
for name, proba in zip(u, pre_proba):
    print(name, "的機率:", proba)

輸入詩:人群 煙火 香檳和氣球 是你 帶我 從派對逃走 逃離 人間 耳語和騷動 這裡 只有 你我和星空 你是 何時 靜靜靠近我 你是 何時 偷偷拯救我 在我 掌心 放了 一顆糖果  人類 為何 都愛看天空 背上 卻都 有一對傷口 生在 這個 凡人的星球 是否 不配 作天使的夢 你將 你的 翅膀給了我 帶我 穿越 狼群和鏡頭 讓我 能夠 品嚐 片刻自由  因為你 所以我 愛上那 片天空 天空下 我在祈求 那是你 牽著我 更多更詳盡歌詞 在 ※ Mojim.com　魔鏡歌詞網 最深刻 的故事 最永恆 的傳說 不過 是你 是我 能夠 平凡生活  因為你 所以我 所以我 不退縮 再不願 這一生中 有遺憾 不反駁 不再聽 別人說 不在乎 誰能懂 只因 世界 再大 不過 你和我 用最小回憶 堆成宇宙  人生 只是 須臾的剎那 人間 只是 天地的夾縫 人們 何苦 要活成修羅 活在 執著 對錯的牢籠 是你 幫我 停下了沙漏 是你 教我 別害怕闖禍 是你 讓我 活得 與眾不同
答案是: 白居易
李白 的機率: 0.026867177175009572
杜甫 的機率: 0.0007903492616228717
白居易 的機率: 0.9723424735633592
